In [3]:
!pip install flask_socketio

     |████████████████████████████████| 56 kB 2.3 MB/s            
     |████████████████████████████████| 52 kB 4.6 MB/s            
You should consider upgrading via the '/Users/ashishgupta/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [6]:
   
from flask import Flask,render_template,redirect,request
from flask.helpers import flash
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from fbprophet import Prophet
import warnings

In [12]:
from werkzeug import debug
warnings.filterwarnings('ignore')
from random import randint
import plotly.graph_objs as go
import plotly.offline as py
import plotly.express as px
from flask_socketio import SocketIO
import datetime
from datetime import timedelta, date
from fbprophet.plot import plot_plotly

In [8]:
from tempfile import TemporaryDirectory

In [9]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)
    

In [13]:
hist_df = pd.read_csv('hkd_data_2010_inr.csv')  
start_date = date(2021, 4, 23)
end_date = datetime.date.today()
df = pd.DataFrame()
for single_date in daterange(start_date, end_date):
    dfs = pd.read_html(f'https://www.xe.com/currencytables/?from=HKD&date={single_date.strftime("%Y-%m-%d")}')[0]
    dfs['Date'] = single_date.strftime("%Y-%m-%d")
    df = df.append(dfs)  
df.to_csv('hkd_data.csv')
inr_df = df[df['Currency'] == 'INR']

nr_df.pop('Rate')<br>
nr_df.pop('Change')

In [14]:
inr_df = df[df['Currency'] == 'INR']


inr_df = pd.concat([hist_df, inr_df], ignore_index=True)


In [15]:
inr_df.head(5)

,Unnamed: 0,Unnamed: 0.1,Change,Currency,Date,HKD per unit,Name,Rate,Units per HKD
0,3.0,3.0,NaN,INR,2011-01-01,0.173893,Indian Rupee,NaN,5.750676
1,166.0,3.0,NaN,INR,2011-01-02,0.173893,Indian Rupee,NaN,5.750676
2,329.0,3.0,NaN,INR,2011-01-03,0.174223,Indian Rupee,NaN,5.739785
3,492.0,3.0,NaN,INR,2011-01-04,0.172422,Indian Rupee,NaN,5.799726
4,655.0,3.0,NaN,INR,2011-01-05,0.171313,Indian Rupee,NaN,5.837257


In [16]:
length=len(inr_df)
data_day1=inr_df[length-1:]
data_day2=inr_df[length-2:length-1]
data_day7=inr_df[length-7:length-6]
data_day15=inr_df[length-15:length-14]
data_day365=inr_df[length-365:length-364]

In [17]:
change_1=float(data_day2['Units per HKD'])-float(data_day1['Units per HKD'])
change_7=float(data_day7['Units per HKD'])-float(data_day1['Units per HKD'])
change_15=float(data_day15['Units per HKD'])-float(data_day1['Units per HKD'])
change_365=float(data_day365['Units per HKD'])-float(data_day1['Units per HKD'])

In [18]:
price_day1=float(data_day1['Units per HKD'])
#print(price_day1,change_1,change_7,change_15,change_365)

In [ ]:
import os
app = Flask("__name__")
app.config["IMAGE_UPLOADS"] = "static/img/"
app.config["Graph_UPLOADS"] = "static/graph/"
app.secret_key = "dadbn2e298ynce8y998c@_shlbsjda"
socketio = SocketIO(app)
@app.route('/')
def index():
    
    
    actual_chart = go.Scatter(y=inr_df["Units per HKD"], x=inr_df["Date"], name= 'Data')
    
    with TemporaryDirectory() as tmp_dir:
        filename = tmp_dir + "tmp.html"
        py.plot([actual_chart],filename = filename , auto_open=False)
        with open(filename, "r") as f:
            graph_html = f.read()
    
    IS_FORECAST = False
    return render_template("step1.html",price_day1=price_day1,change_1=change_1,change_7=change_7,change_15=change_15,change_365=change_365, graph_html=graph_html, IS_FORECAST=IS_FORECAST)
@app.route('/submit',methods=['POST'])
def submit_data():
    try:
        s2=int(request.form['parameter'])
        s1=request.form['options']
    except:
        flash("Please provide valid inputs")
        return redirect("/")
    df= inr_df.drop(['Currency', 'Name', 'HKD per unit'], axis=1)
    df = df.rename(columns={'Units per HKD': 'y', 'Date': 'ds'})

    # to save a copy of the original data..you'll see why shortly. 
    df['y_orig'] = df['y'] 
    # log-transform of y
    df['y'] = np.log(df['y'])
    #instantiate Prophet
    model = Prophet() 
    model.fit(df)
    future_data = model.make_future_dataframe(periods=s2, freq = s1)  #dropdown   
    future_data.tail()
    forecast_data = model.predict(future_data)
    forecast_data[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(5)

    # make sure we save the original forecast data
    forecast_data_orig = forecast_data 
    forecast_data_orig['yhat'] = np.exp(forecast_data_orig['yhat'])
    forecast_data_orig['yhat_lower'] = np.exp(forecast_data_orig['yhat_lower'])
    forecast_data_orig['yhat_upper'] = np.exp(forecast_data_orig['yhat_upper'])
    df['y_log']=df['y'] 
    df['y']=df['y_orig']
    final_df = pd.DataFrame(forecast_data_orig)
    actual_chart = go.Scatter(y=df["y_orig"], name= 'Actual')
    predict_chart = go.Scatter(y=final_df["yhat"], name= 'Predicted')
    predict_chart_upper = go.Scatter(y=final_df["yhat_upper"], name= 'Predicted Upper')
    predict_chart_lower = go.Scatter(y=final_df["yhat_lower"], name= 'Predicted Lower')
    
    with TemporaryDirectory() as tmp_dir:
        filename = tmp_dir + "tmp.html"
        py.plot([actual_chart, predict_chart, predict_chart_upper, predict_chart_lower],filename = filename, auto_open=False)
        with open(filename, "r") as f:
            graph_html = f.read()
    if s1=="D":
        value="Days"
    elif s1=="M":
        value="Month"
    else:
        value="Year"
    final_df_1=final_df[['ds','yhat']].tail(s2)
    final_df_1 = final_df_1.rename(columns={'yhat': 'Units Per HKD', 'ds':value})
    final_df_1.reset_index(drop=True, inplace=True)
    IS_FORECAST = True
    
    table = final_df_1.to_html(classes='table table-striped', border=0)
    table = table.replace('tr style="text-align: right;"', 'tr style="text-align: center;"')
    table = table.replace('<th></th>', '')
    table = table.replace('<th>', '<th colspan="2">', 1)
    print(table)
    return render_template("step1.html",price_day1=price_day1,change_1=change_1,change_7=change_7,change_15=change_15,change_365=change_365, graph_html=graph_html, parameter=s2,table=table, IS_FORECAST = IS_FORECAST)

if __name__ =="__main__":

    socketio.run(app, port=8000, debug=False)

In [24]:
%tb

SystemExit: 1

In [22]:
!pip install gevent-websocket

You should consider upgrading via the '/Users/ashishgupta/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
